In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.Container {width:85% !important;}
div.CodeMirror {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.output_area pre {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.output_wrapper pre {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.input {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
</style>
"""))

# 각 포지션 별 세부 데이터 웹 크롤링

## 후스코어드에서 수비지표 가져오기

In [2]:
# ▶ 1. 필요한 패키지 로드
from selenium import webdriver # 동적 웹 크롤링
import time # 대기시간
import pandas as pd # 데이터 프레임화 해야하기 때문
import numpy as np
from pandas.io.html import read_html

In [123]:
# ▶ 2. 해당 리그 모든 선수의 수비 지표 크롤링
def defensive (url, sleep_time) : # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'D:/Gray_Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)
    
    time.sleep(sleep_time)
    defense = driver.find_element_by_link_text('Defensive')
    defense.click()
    
    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()
    
    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))
    
    df_defensive = pd.DataFrame(columns = ['Player', 'Apps', 'Mins', 'Tackles', 'Inter', 'Fouls',
       'Offsides', 'Clear', 'Drb', 'Blocks', 'OwnG', 'Rating'])
    
    for i in np.arange(total_page) :
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath('//*[@id="statistics-table-defensive"]') 
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')
        
        df2 = read_html(table_html)[0] # 0번 인덱스에 모든 값이 들어가있음
        df_defensive = pd.concat([df_defensive, df2], axis=0)
        
        driver.find_element_by_link_text('next').click() # 크롤링 후 다음페이지로 이동
     
    return(df_defensive)

url = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/PlayerStatistics/Italy-Serie-A-2020-2021'
df = defensive(url, 2)

In [124]:
df.head()

,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player.1
0,"1Marco FestaCrotone, 29, Goalkeeper",1,90,-,-,-,-,1.0,-,-,-,7.80,"Marco Festa Crotone, 29, Goalkeeper"
1,"2Cristiano RonaldoJuventus, 36, M(L),FW",31(2),2803,0.2,0.2,0.5,0.1,0.5,0.4,0.1,-,7.61,"Cristiano Ronaldo Juventus, 36, M(L),FW"
2,"3Zlatan IbrahimovicAC Milan, 39, FW",18(1),1498,0.3,0.3,1.7,0.1,1.4,0.2,0.2,-,7.57,"Zlatan Ibrahimovic AC Milan, 39, FW"
3,"4Lorenzo InsigneNapoli, 30, AM(CL),FW",33(2),2872,0.7,1.1,0.3,-,0.1,1.7,-,-,7.56,"Lorenzo Insigne Napoli, 30, AM(CL),FW"
4,"5Romelu LukakuInter, 28, AM(R),FW",32(4),2887,0.3,0.1,0.9,-,0.9,0.2,-,-,7.47,"Romelu Lukaku Inter, 28, AM(R),FW"


In [125]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1', 'Apps', 'Mins'] , axis=1, inplace=True)

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"1Marco FestaCrotone, 29, Goalkeeper",-,-,-,-,1.0,-,-,-,7.80
1,"2Cristiano RonaldoJuventus, 36, M(L),FW",0.2,0.2,0.5,0.1,0.5,0.4,0.1,-,7.61
2,"3Zlatan IbrahimovicAC Milan, 39, FW",0.3,0.3,1.7,0.1,1.4,0.2,0.2,-,7.57
3,"4Lorenzo InsigneNapoli, 30, AM(CL),FW",0.7,1.1,0.3,-,0.1,1.7,-,-,7.56
4,"5Romelu LukakuInter, 28, AM(R),FW",0.3,0.1,0.9,-,0.9,0.2,-,-,7.47


In [126]:
df.isna().sum()

Player      0
Tackles     0
Inter       0
Fouls       0
Offsides    0
Clear       0
Drb         0
Blocks      0
OwnG        0
Rating      0
dtype: int64

In [127]:
# Player에 이름만 넣기
spl = df['Player'].str.split(',')

print(spl)

0                [1Marco FestaCrotone,  29,  Goalkeeper]
1           [2Cristiano RonaldoJuventus,  36,  M(L), FW]
2                [3Zlatan IbrahimovicAC Milan,  39,  FW]
3             [4Lorenzo InsigneNapoli,  30,  AM(CL), FW]
4                 [5Romelu LukakuInter,  28,  AM(R), FW]
                             ...                        
616          [617Lorenzo Del PintoBenevento,  31,  M(C)]
617       [618Simone ColombiParma Calcio 1913,  30,  GK]
618    [619Chaka TraorèParma Calcio 1913,  16,  Forward]
619                        [620NícolasUdinese,  33,  GK]
620            [621Ciprian TatarusanuAC Milan,  35,  GK]
Name: Player, Length: 621, dtype: object


In [128]:
pls_list = []
for i in range(len(spl)):
    result_a = spl[i][0]
    pls_list.append(result_a)
    
df['Player'] = pls_list

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,1Marco FestaCrotone,-,-,-,-,1.0,-,-,-,7.80
1,2Cristiano RonaldoJuventus,0.2,0.2,0.5,0.1,0.5,0.4,0.1,-,7.61
2,3Zlatan IbrahimovicAC Milan,0.3,0.3,1.7,0.1,1.4,0.2,0.2,-,7.57
3,4Lorenzo InsigneNapoli,0.7,1.1,0.3,-,0.1,1.7,-,-,7.56
4,5Romelu LukakuInter,0.3,0.1,0.9,-,0.9,0.2,-,-,7.47


In [132]:
df.loc[10:30]

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
10,11Theo HernándezAC Milan,1.5,1,1.2,0.2,1.5,0.6,0.2,-,7.28
11,12Papu GómezAtalanta,0.3,0.1,1,-,0.1,0.6,0.1,-,7.27
12,13Hakan CalhanogluAC Milan,0.8,0.5,0.9,-,0.5,0.6,0.2,-,7.27
13,14Luis MurielAtalanta,0.6,0.2,0.5,-,0.1,0.5,-,-,7.25
14,15Federico ChiesaFiorentina,2,0.3,0.7,-,0.7,1.7,0.3,-,7.24
15,16Andrea BelottiTorino,0.9,0.3,1.7,-,0.5,0.7,0.1,-,7.23
16,17Lorenzo PellegriniRoma,1.2,1.3,1.3,-,0.6,1.2,0.1,-,7.23
17,18Junior MessiasCrotone,1.7,1,0.9,-,1.1,1.3,0.3,-,7.22
18,19Federico ChiesaJuventus,1.6,0.4,1,-,0.5,1,-,-,7.21
19,20RafaelSpezia,-,-,-,-,0.3,-,-,-,7.21


In [133]:
engla_teams = ['Man City', 'Liverpool', 'Chelsea', 'Man Utd', 'Arsenal', 'Tottenham', 'Huddersfield', 'Watford',\
              'Bournemouth', 'Crystal Palace', 'Leicester', 'Wolves', 'Everton', 'Cardiff', 'Southampton', 'West Ham',\
              'Fulham', 'Newcastle', 'Brighton', 'Burnley', 'Leeds', 'Norwich', 'Aston Villa', 'Sheff Utd', 'West Brom']

italy_teams = ['Juventus', 'Atalanta', 'AC Milan', 'Udinese', 'Roma', 'Napoli', 'Lazio', 'SPAL', 'Parma Calcio',\
               'Sampdoria', 'Torino', 'Fiorentina', 'Sassuolo', 'Inter', 'Genoa', 'Cagliari', 'Bologna', 'Chievo', 'Empoli',\
               'Frosinone', 'Brescia', 'Lecce', 'Verona', 'Crotone', 'Benevento', 'Spezia']

franc_teams =[]

germa_teams = []

spain_teams = []

In [134]:
#  선수 이름 앞 숫자 자르기, 뒤 팀명 자르기
df['Player'] = df['Player'].replace(r'[0-9]', "", regex=True)

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Marco FestaCrotone,-,-,-,-,1.0,-,-,-,7.80
1,Cristiano RonaldoJuventus,0.2,0.2,0.5,0.1,0.5,0.4,0.1,-,7.61
2,Zlatan IbrahimovicAC Milan,0.3,0.3,1.7,0.1,1.4,0.2,0.2,-,7.57
3,Lorenzo InsigneNapoli,0.7,1.1,0.3,-,0.1,1.7,-,-,7.56
4,Romelu LukakuInter,0.3,0.1,0.9,-,0.9,0.2,-,-,7.47


In [105]:
def eng_players(name):
    for team in engla_teams:
        name = name.replace(team, "")
    return name

In [106]:
def italy_players(name):
    for team in italy_teams:
        name = name.replace(team, "")
    return name

In [135]:
pls_list = []
for player in df['Player']:
    result_b = italy_players(player)
    pls_list.append(result_b)

df['Player'] = pls_list
df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Marco Festa,-,-,-,-,1.0,-,-,-,7.80
1,Cristiano Ronaldo,0.2,0.2,0.5,0.1,0.5,0.4,0.1,-,7.61
2,Zlatan Ibrahimovic,0.3,0.3,1.7,0.1,1.4,0.2,0.2,-,7.57
3,Lorenzo Insigne,0.7,1.1,0.3,-,0.1,1.7,-,-,7.56
4,Romelu Lukaku,0.3,0.1,0.9,-,0.9,0.2,-,-,7.47


In [136]:
# '-'로 처리된 부분 NaN으로 처리하기
df = df.replace("-", "")

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Marco Festa,,,,,1.0,,,,7.80
1,Cristiano Ronaldo,0.2,0.2,0.5,0.1,0.5,0.4,0.1,,7.61
2,Zlatan Ibrahimovic,0.3,0.3,1.7,0.1,1.4,0.2,0.2,,7.57
3,Lorenzo Insigne,0.7,1.1,0.3,,0.1,1.7,,,7.56
4,Romelu Lukaku,0.3,0.1,0.9,,0.9,0.2,,,7.47


In [137]:
# csv파일로 저장하기
df.to_csv('D:/Gray_Bigdata/src/00_Projects/data/spa_1819_defensive_stats.csv')

## 후스코어드에서 미드필드 지표 가져오기

In [ ]:
df.head()

In [ ]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1', 'Apps', 'Mins'] , axis=1, inplace=True)

df.head()

In [ ]:
# Player에 이름만 넣기
spl = df['Player'].str.split(',')

print(spl)

## 후스코어드에서 공격지표 가져오기

In [ ]:
def offensive(url, sleep_time):  # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'C:/Users/swkim/OneDrive/바탕 화면/Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)

    time.sleep(sleep_time)
    offensive = driver.find_element_by_link_text('Offensive')
    offensive.click()

    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()

    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))

    df_offensive = pd.DataFrame(columns=['full_name', 'Apps', 'Mins', 'Goals', 'Assists', 'SpG',
                                         'KeyP', 'Drb', 'Fouled', 'Off', 'Disp', 'UnsTch', 'Rating'])

    for i in np.arange(total_page):
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath(
            '//*[@id="statistics-table-offensive"]')
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')

        df2 = read_html(table_html)[0]  # 0번 인덱스에 모든 값이 들어가있음
        df_offensive = pd.concat([df_offensive, df2], axis=0)

        driver.find_element_by_link_text('next').click()  # 크롤링 후 다음페이지로 이동

    return(df_offensive)


url = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/Stages/16368/PlayerStatistics/England-Premier-League-2018-2019'
df = defensive(url, 2)
